In [1]:
import pandas as pd
df = pd.read_csv("/home/capeie/covid_severity_predictor/final_translated.csv")

In [2]:
df.columns

Index(['Unnamed: 0', 'Gender', 'Hospitalization', 'Death', 'Ventilator',
       'Pneumonia', 'Age', 'Pregnant', 'Diabetes', 'Asthma',
       'Immunosuppression', 'Hypertension', 'Cardiovascular Disease',
       'Obesity ', 'Chronic Renal Insufficiency', 'Tobacco Use',
       'Contact COVID', 'ICU'],
      dtype='object')

In [3]:
df.ICU.unique()

array([ 1, 97,  2, 99])

In [4]:
df = df.drop(["Hospitalization","Ventilator","Death","Unnamed: 0"],axis=1)
index_names = df[ df['ICU'] == 97 ].index 
len(index_names)


3761

In [5]:
target = df['ICU']


In [6]:
target.head()

0     1
1    97
2     2
3    97
4    97
Name: ICU, dtype: int64

In [7]:
x_values = df.drop(['ICU'],axis=1)
x_values = x_values.sort_index(axis=1)

In [8]:
x_values.head()

,Age,Asthma,Cardiovascular Disease,Chronic Renal Insufficiency,Contact COVID,Diabetes,Gender,Hypertension,Immunosuppression,Obesity,Pneumonia,Pregnant,Tobacco Use
0,75,2,2,2,2,1,1,2,2,2,1,2,2
1,22,2,2,2,1,2,2,2,2,2,2,97,2
2,40,2,2,2,2,2,2,2,2,2,1,97,2
3,29,2,2,2,1,2,1,2,2,2,2,2,2
4,61,2,2,2,1,1,1,2,2,1,2,2,2


In [9]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_values['Age'] = min_max_scaler.fit_transform(x_values[['Age']])
x_values.head()


/home/capeie/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


,Age,Asthma,Cardiovascular Disease,Chronic Renal Insufficiency,Contact COVID,Diabetes,Gender,Hypertension,Immunosuppression,Obesity,Pneumonia,Pregnant,Tobacco Use
0,0.765306,2,2,2,2,1,1,2,2,2,1,2,2
1,0.224490,2,2,2,1,2,2,2,2,2,2,97,2
2,0.408163,2,2,2,2,2,2,2,2,2,1,97,2
3,0.295918,2,2,2,1,2,1,2,2,2,2,2,2
4,0.622449,2,2,2,1,1,1,2,2,1,2,2,2


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
column_trans = make_column_transformer((OneHotEncoder(),['Gender','Pneumonia','Pregnant','Diabetes','Asthma','Immunosuppression','Hypertension','Cardiovascular Disease','Obesity ','Chronic Renal Insufficiency','Tobacco Use','Contact COVID']),remainder='passthrough')

In [11]:
y_noob = df.ICU

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
rfc_pipeline = Pipeline(steps = [
    ('data_preprocessing', column_trans),
    ('model', RandomForestClassifier(max_depth = 10,
                                     min_samples_leaf = 3,
                                     min_samples_split = 4,
                                     n_estimators = 200))
])

In [13]:
#train,test,val split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_values, y_noob, random_state = 42)

In [14]:
rfc_pipeline.fit(x_train, y_train)


Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Gender', 'Pneumonia',
                                                   'Pregnant', 'Diabetes',
                                                   'Asthma',
                                                   'Immunosuppression',
                                                   'Hypertension',
                                                   'Cardiovascular Disease',
                                                   'Obesity ',
                                                   'Chronic Renal '
                                                   'Insufficiency',
                                                   'Tobacco Use',
                                                   'Contact COVID'])])),
   

In [15]:
import joblib
joblib.dump(rfc_pipeline, '/home/capeie/covid_severity_predictor/models/rfc_pipeline_ICU.pkl')


['/home/capeie/covid_severity_predictor/models/rfc_pipeline_ICU.pkl']

In [16]:
loaded_model = joblib.load('/home/capeie/covid_severity_predictor/models/rfc_pipeline_ICU.pkl')
pipeline_preds = loaded_model.predict(x_val)

In [17]:
cross_val_score(rfc_pipeline,x_val,y_val,cv=5,scoring='accuracy').mean()


0.8016573939875636

In [18]:
x_new = x_values.sample(1,random_state=99)
pipeline_preds = loaded_model.predict(x_new)


In [19]:
x_new

,Age,Asthma,Cardiovascular Disease,Chronic Renal Insufficiency,Contact COVID,Diabetes,Gender,Hypertension,Immunosuppression,Obesity,Pneumonia,Pregnant,Tobacco Use
115,0.581633,2,2,2,2,2,1,2,2,2,1,2,2


In [20]:
prediction_translate = {1:'Needs ICU',2:'No ICU required',97:'Not Applicable',98:'Ignored',99:'Not Specified'}

In [21]:
prediction_translate[pipeline_preds[0]]

'No ICU required'